In [2]:
import simpy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Simulation parameters
SIM_DURATION = 30 * 12 * 60  # 30 days, 12 hours per day (8 AM to 8 PM)
CAPACITY = 250
WILLINGNESS_TO_VISIT = 0.80  # 80% willingness to visit
TARGET_LOST_PERCENTAGE = 10  # Target 10% lost customers
TOLERANCE = 0.5  # Allowable deviation from the target percentage

# Arrival rates based on the day of the week and time slots
ARRIVAL_RATES = {
    'Sunday': [(8, 12, 15), (12, 17, 50), (17, 20, 50)],
    'Monday': [(8, 12, 15), (12, 17, 50), (17, 20, 50)],
    'Tuesday': [(8, 12, 15), (12, 17, 50), (17, 20, 50)],
    'Wednesday': [(8, 12, 15), (12, 17, 50), (17, 20, 50)],
    'Thursday': [(8, 12, 15), (12, 17, 50), (17, 20, 50)],
    'Friday': [(8, 12, 20), (12, 17, 70), (17, 20, 80)],
    'Saturday': [(8, 12, 20), (12, 17, 70), (17, 20, 80)]
}

# Initialize data for tracking arrivals, inter-arrival times, parking durations, revenue, and parking space utilization
def initialize_data():
    global arrival_pattern, inter_arrival_times, parking_durations, total_revenue, lost_customers, parked_customers, utilization_data
    arrival_pattern = np.zeros((30, 12))  # 30 days, 12 hours per day (8 AM to 8 PM)
    inter_arrival_times = [[] for _ in range(30 * 12)]  # List of lists for storing inter-arrival times for each time slot
    parking_durations = []  # List to store parking durations
    total_revenue = 0  # Total revenue accumulator
    lost_customers = 0  # Lost customers accumulator
    parked_customers = 0  # Parked customers accumulator
    utilization_data = []  # List to store occupancy data for utilization calculation

def get_arrival_rate(day, hour):
    """Return the arrival rate based on the day and hour."""
    for start, end, rate in ARRIVAL_RATES[day]:
        if start <= hour < end:
            return rate
    return 0

def calculate_charge(parking_duration):
    """Calculate the charge based on parking duration with ceiling logic."""
    if parking_duration < 30:
        return 15
    else:
        hours = np.ceil(parking_duration / 60)  # Round up to the nearest hour
        return 25 * hours  # Charge ₹25 per hour

def vehicle(env, parking_lot, day_index, hour_index, inter_arrival_time):
    """Simulates vehicle parking behavior, tracks arrivals, inter-arrival times, and calculates revenue."""
    global total_revenue, lost_customers, parked_customers
    
    # Check if the vehicle decides to park based on willingness to visit
    if np.random.random() > WILLINGNESS_TO_VISIT:
        lost_customers += 1
        return  # Vehicle leaves without parking

    # Generate parking duration (in minutes)
    parking_duration = np.random.exponential(60)  # Exponential distribution with mean 60 minutes
    
    # Store parking duration
    parking_durations.append(parking_duration)
    
    # Calculate the charge based on parking duration
    charge = calculate_charge(parking_duration)
    
    # Update total revenue
    total_revenue += charge
    parked_customers += 1  # Increment parked customers counter
    
    with parking_lot.request() as request:
        yield request
        # Update occupancy data
        current_time = env.now
        num_vehicles = len(parking_lot.users)
        utilization_data.append((current_time, num_vehicles))
        yield env.timeout(parking_duration)
        # Track arrivals for the specific day and hour
        arrival_pattern[day_index, hour_index] += 1  # Use correct indexing for Numpy arrays
        # Store the inter-arrival time
        inter_arrival_times[day_index * 12 + hour_index].append(inter_arrival_time)

def arrival_process(env, parking_lot, day, day_index):
    """Simulates the arrival process based on the day and time."""
    global parked_customers, lost_customers
    while env.now < (day_index + 1) * 12 * 60:  # Simulate only 12 hours per day
        current_hour = (env.now // 60) % 12 + 8  # Convert minutes to simulation hour (8 AM - 8 PM)
        rate = get_arrival_rate(day, current_hour)
        
        # Determine the hour index (0 to 11 for 8 AM to 8 PM)
        hour_index = int(current_hour - 8)  # Adjust index to match 0-11 range
        
        if 0 <= hour_index < 12:  # Ensure hour_index is within valid bounds
            if rate > 0:
                inter_arrival_time = np.random.exponential(60 / rate)  # Convert hourly rate to inter-arrival time
                yield env.timeout(inter_arrival_time)
                env.process(vehicle(env, parking_lot, day_index, hour_index, inter_arrival_time))
            else:
                yield env.timeout(1)  # Wait for 1 minute if no arrival

def run_simulation():
    """Runs the simulation for 30 days."""
    global total_revenue, lost_customers, parked_customers
    env = simpy.Environment()
    parking_lot = simpy.Resource(env, capacity=CAPACITY)

    for day_index in range(30):
        day_name = list(ARRIVAL_RATES.keys())[day_index % 7]  # Cycle through days of the week
        env.process(arrival_process(env, parking_lot, day_name, day_index))

        # Simulate 12 hours of the current day
        env.run(until=(day_index + 1) * 12 * 60)

def adjust_arrival_rates(percentage_lost_customers):
    """Adjust arrival rates based on the percentage of lost customers."""
    adjustment_factor = 1.05 if percentage_lost_customers < TARGET_LOST_PERCENTAGE else 0.95

    # Update arrival rates based on the adjustment factor
    for day in ARRIVAL_RATES:
        updated_slots = []
        for start, end, rate in ARRIVAL_RATES[day]:
            updated_slots.append((start, end, rate * adjustment_factor))
        ARRIVAL_RATES[day] = updated_slots

# Dynamic adjustment loop
acceptable_range = (TARGET_LOST_PERCENTAGE - TOLERANCE, TARGET_LOST_PERCENTAGE + TOLERANCE)
while True:
    initialize_data()  # Reset all data accumulators
    run_simulation()  # Run the parking lot simulation

    # Calculate percentage of lost customers
    total_attempted_arrivals = parked_customers + lost_customers
    percentage_lost_customers = (lost_customers / total_attempted_arrivals) * 100 if total_attempted_arrivals > 0 else 0

    # Calculate average parking duration
    avg_parking_duration = np.mean(parking_durations) if parking_durations else 0

    # Calculate average inter-arrival time
    all_inter_arrival_times = [time for slot in inter_arrival_times for time in slot]  # Flatten list of lists
    avg_inter_arrival_time = np.mean(all_inter_arrival_times) if all_inter_arrival_times else 0

    # Calculate total parking space utilization
    time_slot_utilization = {'8-11:59': [], '12-16:59': [], '17-20:00': []}
    total_time_per_slot = {'8-11:59': 4 * 60 * 30, '12-16:59': 5 * 60 * 30, '17-20:00': 3 * 60 * 30}  # Total time per slot

    # Utility function to get time slot based on the hour
    def get_time_slot(hour):
        if 8 <= hour < 12:
            return '8-11:59'
        elif 12 <= hour < 17:
            return '12-16:59'
        elif 17 <= hour < 20:
            return '17-20:00'
        return None

    # Compute the total occupied time based on occupancy data and categorize by time slots
    previous_time = 0
    for current_time, num_vehicles in utilization_data:
        hour = int((current_time // 60) % 12) + 8  # Convert minutes to simulation hour (8 AM - 8 PM)
        time_slot = get_time_slot(hour)
        if time_slot:
            time_spent = current_time - previous_time
            time_slot_utilization[time_slot].append(time_spent * (num_vehicles / CAPACITY))
        previous_time = current_time

    # Calculate average utilization for each time slot
    average_utilization_by_slot = {
        slot: (np.sum(time_slot_utilization[slot]) / total_time_per_slot[slot]) * 100
        for slot in time_slot_utilization
    }

    # Print relevant factors
    print(f"\nSimulation Results after Adjusting Arrival Rates:")
    print(f"Current Lost Customer Percentage: {percentage_lost_customers:.2f}%")
    print(f"Total Revenue: ₹{total_revenue:.2f}")
    print(f"Average Parking Duration: {avg_parking_duration:.2f} minutes")
    print(f"Average Inter-Arrival Time: {avg_inter_arrival_time:.2f} minutes")
    print(f"Total Parked Customers: {parked_customers}")
    print(f"Total Lost Customers: {lost_customers}")

    # Print average parking space utilization by time slot
    print("\nAverage Parking Space Utilization by Time Slot:")
    for slot, utilization in average_utilization_by_slot.items():
        print(f"{slot}: {utilization:.2f}%")
    
    # Check if the lost customer percentage is within the acceptable range
    if acceptable_range[0] <= percentage_lost_customers <= acceptable_range[1]:
        print(f"Lost customer percentage is within the acceptable range: {percentage_lost_customers:.2f}%")
        break
    else:
        print(f"Adjusting arrival rates...")
        adjust_arrival_rates(percentage_lost_customers)



Simulation Results after Adjusting Arrival Rates:
Current Lost Customer Percentage: 19.89%
Total Revenue: ₹440270.00
Average Parking Duration: 60.14 minutes
Average Inter-Arrival Time: 1.39 minutes
Total Parked Customers: 12336
Total Lost Customers: 3063

Average Parking Space Utilization by Time Slot:
8-11:59: 8.15%
12-16:59: 15.04%
17-20:00: 18.64%
Adjusting arrival rates...

Simulation Results after Adjusting Arrival Rates:
Current Lost Customer Percentage: 19.44%
Total Revenue: ₹427190.00
Average Parking Duration: 59.93 minutes
Average Inter-Arrival Time: 1.45 minutes
Total Parked Customers: 11991
Total Lost Customers: 2894

Average Parking Space Utilization by Time Slot:
8-11:59: 8.32%
12-16:59: 14.43%
17-20:00: 17.83%
Adjusting arrival rates...

Simulation Results after Adjusting Arrival Rates:
Current Lost Customer Percentage: 19.97%
Total Revenue: ₹393980.00
Average Parking Duration: 59.90 minutes
Average Inter-Arrival Time: 1.57 minutes
Total Parked Customers: 11092
Total Los